In [8]:
import os
import sys
import pandas as pd
import logging
from google.cloud import bigquery
from hashlib import md5
from typing import List
import uuid
from pandas.io.json import json_normalize
import pycountry

In [19]:
#creating dim_country
df = pd.read_csv('./data/Emissions_by_Country_2002-2022.csv')
lat_long_df = pd.read_csv('./data/world_country_and_usa_states_latitude_and_longitude_values.csv')
df1 = df[['Country', 'ISO 3166-1 alpha-3']]

#make a 2 letter country code
def alpha3code(column):
    CODE=[]
    for country in column:
        try:
            code=pycountry.countries.get(name=country).alpha_3
           # .alpha_3 means 3-letter country code 
           # .alpha_2 means 2-letter country code
            CODE.append(code)
        except:
            CODE.append('None')
    return CODE
    
#make a 3 digit country code column in my lat_long_df
lat_long_df['long_code']=alpha3code(lat_long_df.country)
lat_long_df = lat_long_df[['country','long_code','country_code','latitude','longitude']]
lat_long_df = lat_long_df.rename(columns={'country_code': 'country_code_2', 'long_code': 'country_code'})

#read country size csv
size_df = pd.read_csv('./data/country_size.csv', dtype=str)
size_df = size_df[['Country Code','2020']]
size_df["2020"] = size_df['2020'].astype('float').round(decimals=0)
size_df = size_df.rename(columns={'2020': 'land_area_square_km', 'Country Code': 'country_code'})

#merge datasets to create dim_countries
dim_country = pd.merge(lat_long_df,size_df, how='inner', on='country_code')
dim_country = dim_country[['country', 'country_code','country_code_2','land_area_square_km','latitude','longitude']]
dim_country["created_at"] = dim_country.apply(lambda _: ' ', axis=1)
dim_country["modified_at"] = dim_country.apply(lambda _: ' ', axis=1)
display(dim_country)



,country,country_code,country_code_2,land_area_square_km,latitude,longitude,created_at,modified_at
0,Andorra,AND,AD,470.0,42.546245,1.601554,,
1,United Arab Emirates,ARE,AE,71020.0,23.424076,53.847818,,
2,Afghanistan,AFG,AF,652230.0,33.939110,67.709953,,
3,Antigua and Barbuda,ATG,AG,440.0,17.060816,-61.796428,,
4,Albania,ALB,AL,27400.0,41.153332,20.168331,,
...,...,...,...,...,...,...,...,...
181,Samoa,WSM,WS,2780.0,-13.759029,-172.104629,,
182,Yemen,YEM,YE,527970.0,15.552727,48.516388,,
183,South Africa,ZAF,ZA,1213090.0,-30.559482,22.937506,,
184,Zambia,ZMB,ZM,743390.0,-13.133897,27.849332,,
